In [1]:
import requests
import json
from datetime import datetime
import zipfile

In [41]:
url = 'https://donnees.roulez-eco.fr/opendata/instantane'
response = requests.get(url)


if response.status_code == 200:
    zip_path = './instantanes_files/zip/instantane_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.zip'
    with open(zip_path, 'wb') as f:
        f.write(response.content)
        print('File saved successfully.')

    output_folder = './instantanes_files/xml/'

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_folder)
        

    print('File unzipped successfully.')
else:
    print('Failed to download the file.')

File saved successfully.
File unzipped successfully.


In [45]:
# !pip install xmltodict
#xml to json
import xmltodict
import json
import os

xml_folder = './instantanes_files/xml/'
json_folder = './instantanes_files/json/'

for filename in os.listdir(xml_folder):
    if filename.endswith(".xml"):
        xml_path = xml_folder + filename
        json_path = json_folder + filename.replace('.xml', '.json')
        with open(xml_path, 'r', encoding='ISO-8859-1') as xml_file:
            xml_content = xml_file.read()
            json_content = json.dumps(xmltodict.parse(xml_content), indent=4)
            with open(json_path, 'w') as json_file:
                json_file.write(json_content)
                print('File ' + filename + ' converted successfully.')
    else:
        continue

File PrixCarburants_instantane.xml converted successfully.


In [1]:
import xml.etree.ElementTree as ET
import psycopg2

In [5]:
# Parse XML file
tree = ET.parse('instantanes_files/xml/PrixCarburants_instantane.xml')
root = tree.getroot()

In [8]:
# Establish database connection
conn = psycopg2.connect(
    database="carburants",
    user="yzpt",
    password="yzpt",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

In [9]:
# Create table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS table_from_xml (
        record_timestamp TIMESTAMP,
        id BIGINT,
        latitude REAL,
        longitude REAL,
        cp VARCHAR(50),
        pop TEXT,
        adresse TEXT,
        ville VARCHAR(50),
        horaires TEXT,
        services TEXT,
        gazole_maj TIMESTAMP,
        gazole_prix REAL,
        sp95_maj TIMESTAMP,
        sp95_prix REAL,
        e85_maj TIMESTAMP,
        e85_prix REAL,
        gplc_maj TIMESTAMP,
        gplc_prix REAL,
        e10_maj TIMESTAMP,
        e10_prix REAL,
        sp98_maj TIMESTAMP,
        sp98_prix REAL,
        carburants_disponibles VARCHAR(50),
        carburants_indisponibles VARCHAR(50),
        horaires_automate_24_24 TEXT,
        departement VARCHAR(50),
        code_departement VARCHAR(50),
        region VARCHAR(50),
        code_region VARCHAR(50),
        PRIMARY KEY (record_timestamp, id)
        );
''')
conn.commit()

In [18]:
for item in root.findall('pdv')[:3]:
    print(item.get('id'))



18230001
54300001
45450003


In [22]:
# Insert data into the database
for item in root.findall('pdv'):
    id = int(item.get('id'))
    latitude = float(item.get('latitude'))
    longitude = float(item.get('longitude'))
    print('type(id): ', type(id), 'type(latitude): ', type(latitude), 'type(longitude): ', type(longitude))

    # Insert data into the table
    cursor.execute('''
        INSERT INTO table_from_xml (id, latitude, longitude)
        VALUES (%s, %s, %s);
        ''', 
        (id, latitude, longitude))

conn.commit()

# Close the database connection
cursor.close()
conn.close()


type(id):  <class 'int'> type(latitude):  <class 'float'> type(longitude):  <class 'float'>


InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block
